In [18]:
import pandas as pd
import csv
from statistics import median
from projects import project_list

In [19]:
r_file_name = 'static_baseline_batching.csv'
result_file = open(r_file_name, 'w')
result_headers = ['project', 'algorithm', 'batch_size', 'builds_saved', 'median_delay', 'testall_size']
writer = csv.writer(result_file)
writer.writerow(result_headers)

69

In [20]:
batch_total = 0

In [21]:
def output_values(tr_results):
    l = []
    i = 0
    while i < len(tr_results):
        if tr_results[i] == 'passed':
            l.append(1)
        else:
            l.append(0)
        i += 1
    
    return l

In [22]:
def batch_bisect(batch_results):
    global batch_total
    
    batch_total += 1
    
    if len(batch_results) == 1:
        return
    
    if 0 in batch_results:
        half_batch = len(batch_results)//2
        batch_bisect(batch_results[:half_batch])
        batch_bisect(batch_results[half_batch:])

In [23]:
def batch_stop_4(batch_results):
    global batch_total
    
    batch_total += 1
    
    if len(batch_results) <= 4:
        if 0 in batch_results:
            batch_total += 4
        return
    
    if 0 in batch_results:
        half_batch = len(batch_results)//2
        batch_stop_4(batch_results[:half_batch])
        batch_stop_4(batch_results[half_batch:])

In [24]:
def bb_batching(project, data):
    global batch_total
    
    p_name = project.split('/')[1]
    
    result_file = open('static_baseline_batching.csv', 'a+')
    writer = csv.writer(result_file)
    
    results = output_values(data['tr_status'].tolist())
    length = len(results)
    
    batch_sizes = [2, 4, 8, 16]
    for b in batch_sizes:
        builds = 0
        delays = []
        
        i = 0
        while i < length:
            if length - i > b:
                batch = results[i:i+b]
                delays.extend([b-x-1 for x in range(b)])
            else:
                batch = results[i:]
                delays.extend([b-x-1 for x in range(length-i)])
            batch_total = 0
            batch_bisect(batch)
            builds += batch_total
            
            i += b
        
        builds_saved = 100-(100*builds/length)
        median_delay = median(delays)
        
        writer.writerow([p_name, 'BATCHBISECT', b, builds_saved, median_delay, length])

In [25]:
def bs4_batching(project, data):
    global batch_total
    
    p_name = project.split('/')[1]
    
    result_file = open('static_baseline_batching.csv', 'a+')
    writer = csv.writer(result_file)
    
    results = output_values(data['tr_status'].tolist())
    length = len(results)
    
    batch_sizes = [8, 16]
    for b in batch_sizes:
        builds = 0
        delays = []
        
        i = 0
        while i < length:
            if length - i > b:
                batch = results[i:i+b]
                delays.extend([b-x-1 for x in range(b)])
            else:
                batch = results[i:]
                delays.extend([b-x-1 for x in range(length-i)])
            batch_total = 0
            batch_stop_4(batch)
            builds += batch_total
            
            i += b
        
        builds_saved = 100-(100*builds/length)
        median_delay = median(delays)
        
        writer.writerow([p_name, 'BATCHSTOP4', b, builds_saved, median_delay, length])

In [26]:
discard_list = []
for p in project_list:
    data = pd.read_csv('data/25_1_travis_data/'+p)
    
    branch_type = data['git_branch'].tolist()
    if 'master' in branch_type:
        data = data[ data['git_branch'] == 'master']
    else:
        data = data[ data['git_branch'] == 'trunk']
    
    
    data = data.iloc[100:]
    if len(data) == 0:
        print('Could not continue for {}'.format(p))
        discard_list.append(p)
        continue
    
    bb_batching(p, data)
    bs4_batching(p, data)
    

Could not continue for BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv
Could not continue for RS485-LogisticsPipes/RS485-LogisticsPipes.csv
Could not continue for datastax-java-driver/datastax-java-driver.csv


In [27]:
discard_list

['BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv',
 'RS485-LogisticsPipes/RS485-LogisticsPipes.csv',
 'datastax-java-driver/datastax-java-driver.csv']